In [1]:
import cv2
import numpy as np
from cv_bridge import CvBridge
from scipy.ndimage import rotate
import matplotlib.pyplot as plt
import timeit
import ransac


In [2]:
class image_processor:
    equirec_i = np.load('/home/shane/stitching/equirec_i.npy')
    equirec_j = np.load('/home/shane/stitching/equirec_j.npy')

    circular_i = np.load('/home/shane/stitching/circular_i.npy')
    circular_j = np.load('/home/shane/stitching/circular_j.npy')

    sift = cv2.SIFT_create()

    def __init__(self, image):
        self.img = image
        self.spherical_img = self.to_spherical(image)
        self.circular_img = self.to_circular(self.spherical_img)

    def to_spherical(self, img):
        return img[self.equirec_i, self.equirec_j]

    def to_circular(self, spherical_img):
        return spherical_img[self.circular_i, self.circular_j]
        
    def get_features(self):
        grayscale_img = cv2.cvtColor(self.circular_img, cv2.COLOR_BGR2GRAY)
        kp, des = self.sift.detectAndCompute(grayscale_img, None)
        return kp, des

    def show_features(self):
        circular_img = np.copy(self.circular_img)
        kp, des = self.get_features()

        for keypoint in kp:
            point = keypoint.pt
            cv2.circle(circular_img, (int(point[0]), int(point[1])), 5, (200,0,0),2)

        cv2.imshow('Features', circular_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [3]:
class model_estimator:    
    def transform_point(self, points, matrix):
        new_points = cv2.transform(points.T[:, :, np.newaxis].T, matrix)[0].T
        new_points = np.divide(new_points, new_points[2]).astype(int)
        return new_points[:2].T

    def fit(self, data):
        data = data[0:4]
        pts1 = np.float32(data[:, 0])
        pts2 = np.float32(data[:, 1])
        return cv2.getPerspectiveTransform(pts2, pts1)

    def get_error(self, data, model):
        center = data[:, 0]
        right = self.transform_point(data[:, 1], model)

        return np.linalg.norm(center - right, axis=1) 

In [4]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

class image_combiner:
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    estimator = model_estimator()

    def __init__(self, image0, image1):
        self.ip0 = image_processor(image0)
        self.ip1 = image_processor(image1)

    def match_features(self):
        kp_query, des_query = self.ip0.get_features()
        kp_train, des_train = self.ip1.get_features()

        matches = self.flann.knnMatch(des_query, des_train, k=2)
        query_pixels, train_pixels = zip(*[(kp_query[m.queryIdx].pt, kp_train[m.trainIdx].pt) for (m, n) in matches if m.distance < 0.6 * n.distance])
        return np.stack((query_pixels, train_pixels), axis=1)

    def estimate_model(self):
        matches = self.match_features()
        return ransac.ransac(matches, self.estimator, 4, 100, 10, len(matches)*0.75, debug=False)

    def combine(self):
        matrix = self.estimate_model()
        img2 = cv2.warpPerspective(self.ip1.circular_img, matrix, (1280, 1280))
        img1 = self.ip0.circular_img

        combine_img = np.zeros((1280, 1280, 3), np.uint8) 
        mask = np.all(img1 <= [5,5,5], axis=2)
        combine_img[np.invert(mask)] = img1[np.invert(mask)]
        combine_img[mask] = img2[mask]
        
        # dst = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)
        img2[mask] = [0,0,0]
        cv2.imshow('Features', combine_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [5]:
img0 = cv2.imread('/home/shane/camera0.jpg')
img1 = cv2.imread('/home/shane/camera1.jpg')
img2 = cv2.imread('/home/shane/camera2.jpg')
img3 = cv2.imread('/home/shane/camera3.jpg')
img4 = cv2.imread('/home/shane/camera4.jpg')

ip0 = image_processor(img0)
ip1 = image_processor(img1)
ip2 = image_processor(img2)
ip3 = image_processor(img3)
ip4 = image_processor(img4)

In [38]:
cv2.imwrite('/home/shane/camera0_cir.jpg', ip0.circular_img[210: 1070, 210:1070])

True

In [17]:
cv2.imshow('Features', ip4.circular_img[160: 1120, 160:1120])
cv2.waitKey(0)
cv2.destroyAllWindows()